### Author: Vidhi Kokel
### Date: 09/05/2022
#### This notebook detects and classifies the aircrafts from the provided image using Yolov5 model.
#### Here, the whole dataset is used for the training and validation.

### Installing and importing libraries

In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
!pip install numpy==1.17
!pip install PyYAML==5.3.1
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip install sklearn

  Using cached numpy-1.17.0.zip (6.5 MB)
  Running setup.py clean for numpy
Failed to build numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
    Running setup.py install for numpy: started
    Running setup.py install for numpy: finished with status 'error'
  Rolling back uninstall of numpy
  Moving to c:\programdata\anaconda3\envs\dl\lib\site-packages\numpy-1.22.3.dist-info\
   from C:\ProgramData\Anaconda3\envs\dl\Lib\site-packages\~umpy-1.22.3.dist-info
  Moving to c:\programdata\anaconda3\envs\dl\lib\site-packages\numpy\
   from C:\ProgramData\Anaconda3\envs\dl\Lib\site-packages\~umpy
  Moving to c:\programdata\anaconda3\envs\dl\scripts\f2py.exe
   from C:\Users\vidhi\AppData\Local\Temp\pip-uninstall-r2s1_ik4\f2py.exe


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\envs\dl\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\vidhi\\AppData\\Local\\Temp\\pip-install-koyi8nht\\numpy_a6abf2a584b64e77b2a19f03bb44fe3b\\setup.py'"'"'; __file__='"'"'C:\\Users\\vidhi\\AppData\\Local\\Temp\\pip-install-koyi8nht\\numpy_a6abf2a584b64e77b2a19f03bb44fe3b\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\vidhi\AppData\Local\Temp\pip-wheel-6vtlyk0r'
       cwd: C:\Users\vidhi\AppData\Local\Temp\pip-install-koyi8nht\numpy_a6abf2a584b64e77b2a19f03bb44fe3b\
  Complete output (293 lines):
  Running from numpy source directory.
  blas_opt_info:
  blas_mkl_info:
  No module named 'numpy.distutils._ms

  Cloning https://github.com/cocodataset/cocoapi.git to c:\users\vidhi\appdata\local\temp\pip-req-build-qx24i6ax
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Running setup.py clean for pycocotools
Failed to build pycocotools
    Running setup.py install for pycocotools: started
    Running setup.py install for pycocotools: finished with status 'error'


  Running command git clone -q https://github.com/cocodataset/cocoapi.git 'C:\Users\vidhi\AppData\Local\Temp\pip-req-build-qx24i6ax'
  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\envs\dl\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\vidhi\\AppData\\Local\\Temp\\pip-req-build-qx24i6ax\\PythonAPI\\setup.py'"'"'; __file__='"'"'C:\\Users\\vidhi\\AppData\\Local\\Temp\\pip-req-build-qx24i6ax\\PythonAPI\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\vidhi\AppData\Local\Temp\pip-wheel-inxt9j8f'
       cwd: C:\Users\vidhi\AppData\Local\Temp\pip-req-build-qx24i6ax\PythonAPI
  Complete output (16 lines):
  running bdist_wheel
  running build
  running build_py
  cr

In [3]:
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

fatal: destination path 'apex' already exists and is not an empty directory.


### Downloading dataset

In [4]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/a2015003713/militaryaircraftdetectiondataset")

Skipping, found downloaded files in ".\militaryaircraftdetectiondataset" (use force=True to force download)


In [5]:
from pathlib import Path
import numpy as np
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10
np.random.seed(42)

import pandas as pd
import glob
import os

C:\ProgramData\Anaconda3\envs\dl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Identifying total number of classes in the provided dataset

In [6]:
# setting the path for joining multiple files
dataset_path = "militaryaircraftdetectiondataset\\dataset\\"
files = os.path.join(dataset_path, "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
                               filename  width  height   class  xmin  ymin  \
0      000106393cfe2343888c584e65fd2274   3520    2514     F16     1     1   
1      000ec980b5b17156a55093b4bd6004ab   1000     551   Be200   308   126   
2      0036c2784a5b8b2a4a1a4bd9109eb2f7   2048    1328  EF2000   726   594   
3      0041e69431bf872309d1aff628b6494f   1279     725      E2   217   281   
4      0048b6531d2792ef6f5b7235b357a348    960    1200  Rafale   340   303   
...                                 ...    ...     ...     ...   ...   ...   
10004  ffa565e4f573979207ea525e63c204d5   3000    2209     F18   399  1311   
10005  ffa565e4f573979207ea525e63c204d5   3000    2209     F18  1840   480   
10006  ffa8d739b58bbf6b65e6d5dd9b2e8a69   1800    1177      B1   108   417   
10007  ffca28378a5df5113d498f59ed282a98   1244     960     F16     2    91   
10008  ffff0595d9b782e9cd3c537529bf6027   1200     800    Su57   322    

In [7]:
categories = list(set(df["class"]))
categories.sort()
print(categories)
print(len(categories))

['A10', 'A400M', 'AG600', 'B1', 'B2', 'B52', 'Be200', 'C130', 'C17', 'C5', 'E2', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'J20', 'JAS39', 'MQ9', 'Mig31', 'Mirage2000', 'RQ4', 'Rafale', 'SR71', 'Su34', 'Su57', 'Tornado', 'Tu160', 'Tu95', 'U2', 'US2', 'V22', 'Vulcan', 'XB70', 'YF23']
39


### Preparing training and validation data

In [8]:
train_data, val_data = train_test_split(files, test_size=0.3)
len(train_data), len(val_data)

(4020, 1723)

In [9]:
def convert_csv_to_text(file, path):
    import csv
    csv_file = file
    txt_file = file.split('\\')[2][:-4] + '.txt'
    with open(os.path.join(path, txt_file), "w+") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            reader = csv.reader(my_input_file)
            next(reader, None)
            for row in reader:
                category_idx = categories.index(row[3])
                xmin, ymin = int(row[4]), int(row[5])
                xmax, ymax = int(row[6]), int(row[7])
                bbox_width, bbox_height = int(row[1]), int(row[2])
                my_output_file.write(
                    f"{category_idx} {((xmin + xmax) / 2) / bbox_width} {((ymin + ymax) / 2) / bbox_height} {bbox_width/bbox_width} {bbox_height/bbox_height}\n"
                )
    my_input_file.close()
    my_output_file.close()
    return

In [10]:
!mkdir "{dataset_path}images\\train_data"
!mkdir "{dataset_path}labels\\train_data"
!mkdir "{dataset_path}images\\val_data"
!mkdir "{dataset_path}labels\\val_data"

In [11]:
import shutil
for i in train_data:
    if i not in "militaryaircraftdetectiondataset\\dataset\\labels\\train_data":
        convert_csv_to_text(i, "militaryaircraftdetectiondataset\\dataset\\labels\\train_data")
        shutil.copy2(i[:-4] + ".jpg", "militaryaircraftdetectiondataset\\dataset\\images\\train_data")
for i in val_data:
    if i not in "militaryaircraftdetectiondataset\\dataset\\labels\\val_data":
        convert_csv_to_text(i, "militaryaircraftdetectiondataset\\dataset\\labels\\val_data")
        shutil.copy2(i[:-4] + ".jpg", "militaryaircraftdetectiondataset\\dataset\\images\\val_data")

In [12]:
f = open("detection.yaml", "w")
f.write("train: ../militaryaircraftdetectiondataset/dataset/images/train_data/")
f.write("\nval: ../militaryaircraftdetectiondataset/dataset/images/val_data/")
f.write("\nnc: " + str(len(categories)))
f.write("\nnames: [" + ', '.join("'" + str(i) + "'" for i in categories) + "]")
f.close()

#open and read the file after the appending:
f = open("detection.yaml", "r")
print(f.read())
f.close()

train: ../militaryaircraftdetectiondataset/dataset/images/train_data/
val: ../militaryaircraftdetectiondataset/dataset/images/val_data/
nc: 39
names: ['A10', 'A400M', 'AG600', 'B1', 'B2', 'B52', 'Be200', 'C130', 'C17', 'C5', 'E2', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'J20', 'JAS39', 'MQ9', 'Mig31', 'Mirage2000', 'RQ4', 'Rafale', 'SR71', 'Su34', 'Su57', 'Tornado', 'Tu160', 'Tu95', 'U2', 'US2', 'V22', 'Vulcan', 'XB70', 'YF23']


### Installing wandb for tracking Yolov5 execution.

In [13]:
# A working account of wandb is required for tracking the execution
# !pip install wandb
# import wandb
# wandb.login()

wandb: Currently logged in as: vidhik. Use `wandb login --relogin` to force relogin


True

### Model training and validation

In [14]:
!python yolov5/train.py --img 640 --batch 8 --epochs 3 \
  --data detection.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5/yolov5s.pt \
  --name yolov5s_detection

wandb: Currently logged in as: vidhik. Use `wandb login --relogin` to force relogin
train: weights=yolov5/yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=detection.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=yolov5s_detection, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v6.1-177-gd059d1d torch 1.11.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0,

In [18]:
!python yolov5/detect.py --weights yolov5/runs/train/yolov5s_detection6/weights/best.pt --img 640 --conf 0.4 --source militaryaircraftdetectiondataset/dataset/images/val_data

detect: weights=['yolov5/runs/train/yolov5s_detection3/weights/best.pt'], source=militaryaircraftdetectiondataset/dataset/images/val_data, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-177-gd059d1d torch 1.11.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7115308 parameters, 0 gradients, 16.1 GFLOPs
image 1/1723 C:\Users\vidhi\ML\DL_in_CV\Project\militaryaircraftdetectiondataset\dataset\images\val_data\00a146155690879cedf0b20325b0ba18.jpg: 480x640 Done. (0.140s)
image 2/1723 C:\Users\vidhi\ML\DL_in_CV\Project\militaryaircraftdetectiondataset\dataset\images\val_data\00c09f406d31a0cd9402862fbd26d930.jpg: 448x640 Done. (0.1